In [79]:
import h3
from h3 import Polygon
import geopandas as gpd
from shapely.geometry import Polygon as ShapelyPolygon
import folium

def get_argentina_hexagons_gdf(resolution):
    # Get Argentina's shape
    world = gpd.read_file(gpd.datasets.get_path('naturalearth_lowres'))
    argentina = world[world.name == 'Argentina']
    geom = argentina.geometry.iloc[0]

    # Get hexagons
    all_hexagons = set()
    if geom.geom_type == 'MultiPolygon':
        polygons = list(geom.geoms)
    else:
        polygons = [geom]

    for polygon in polygons:
        if hasattr(polygon, 'exterior'):
            coords = list(polygon.exterior.coords)
            vertices = [(y, x) for x, y in coords]
            h3_polygon = Polygon(vertices)
            hexagons = h3.polygon_to_cells(h3_polygon, resolution)
            all_hexagons.update(hexagons)

    # Create GeoDataFrame directly from h3.cell_to_boundary but swap lat/lng
    geometries = [ShapelyPolygon([(lng, lat) for lat, lng in h3.cell_to_boundary(h)]) for h in all_hexagons]
    gdf = gpd.GeoDataFrame(
        index=list(all_hexagons),
        geometry=geometries,
        crs="EPSG:4326"
    )

    return gdf

# Create GeoDataFrame
resolution = 4
gdf = get_argentina_hexagons_gdf(resolution)
gdf.head()

/var/folders/3b/q__cv4rj3gl3n9j3lyvf0fdh0000gn/T/ipykernel_1374/4117665116.py:9: FutureWarning: The geopandas.dataset module is deprecated and will be removed in GeoPandas 1.0. You can get the original 'naturalearth_lowres' data from https://www.naturalearthdata.com/downloads/110m-cultural-vectors/.
  world = gpd.read_file(gpd.datasets.get_path('naturalearth_lowres'))


,geometry
84cfa2bffffffff,"POLYGON ((-68.93974 -50.54134, -68.74315 -50.3..."
84cea07ffffffff,"POLYGON ((-71.26975 -43.36204, -71.08761 -43.1..."
84b34e1ffffffff,"POLYGON ((-62.72670 -24.35562, -62.91615 -24.5..."
84b34a1ffffffff,"POLYGON ((-62.78079 -25.39658, -62.97111 -25.5..."
84c36c5ffffffff,"POLYGON ((-67.17063 -35.74978, -67.38116 -35.8..."


In [80]:
gdf.to_file("/Users/santi/DataspellProjects/Tesina/Scripts/Data/Geo/Hexagonos_H3_Argentina_Resolution_4.geojson")

In [81]:
# Create a map centered on Argentina
m = folium.Map(location=[-38.416097, -63.616672], zoom_start=4)

# Add the hexagons to the map
folium.GeoJson(
    gdf,
    style_function=lambda x: {
        'fillColor': '#3186cc',
        'color': '#000000',
        'fillOpacity': 0.3,
        'weight': 1,
    }
).add_to(m)

# Add Argentina's boundaries for reference
folium.GeoJson(
    gpd.read_file(gpd.datasets.get_path('naturalearth_lowres'))[
        lambda df: df.name == 'Argentina'
    ],
    style_function=lambda x: {
        'fillColor': 'transparent',
        'color': 'red',
        'weight': 2
    }
).add_to(m)

# Save the map
m

/var/folders/3b/q__cv4rj3gl3n9j3lyvf0fdh0000gn/T/ipykernel_1374/1219547056.py:17: FutureWarning: The geopandas.dataset module is deprecated and will be removed in GeoPandas 1.0. You can get the original 'naturalearth_lowres' data from https://www.naturalearthdata.com/downloads/110m-cultural-vectors/.
  gpd.read_file(gpd.datasets.get_path('naturalearth_lowres'))[
